In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import yaml
import warnings
import sys
from ultralytics import YOLO
from pathlib import Path
warnings.filterwarnings('ignore')

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 환경에 따라 PROJECT_ROOT 설정
if IN_COLAB:
    PROJECT_ROOT = Path("/content/drive/MyDrive/likelion/PCB")
else:
    PROJECT_ROOT = Path.cwd().parent

# sys.path 추가 (import용)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 작업 디렉토리 변경 (파일 접근용) - 중요!
os.chdir(PROJECT_ROOT)

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Current working directory: {os.getcwd()}")

Environment: Local
PROJECT_ROOT: /Users/mungughyeon/내 드라이브/likelion/PCB
Current working directory: /Users/mungughyeon/내 드라이브/likelion/PCB


In [4]:
!nvidia-smi

Wed Jan 14 09:22:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
from src.utils.config import load_config

# 파라미터 확인
config_path = f"{PROJECT_ROOT}/configs/config.yaml"
config = load_config(config_path)
config

{'model': {'type': 'yolo11',
  'size': 'm',
  'pretrained': False,
  'checkpoint': 'weights/best_yolov11m_imgsz1280.pt'},
 'data': {'data_yaml': 'dataset/roboflow/data.yaml', 'img_size': 1280},
 'training': {'epochs': 30,
  'batch_size': 32,
  'freeze': 10,
  'device': 'cuda',
  'workers': 8,
  'multi_scale': True,
  'mosaic': 0.1,
  'mixup': 0.1,
  'box': 7.5,
  'cls': 0.5,
  'optimizer': 'AdamW',
  'lr0': 0.0002,
  'lrf': 0.01,
  'momentum': 0.937,
  'weight_decay': 0.0005,
  'patience': 10,
  'save_period': -1,
  'save_dir': 'outputs'},
 'wandb': {'enabled': True, 'project': 'PCB_Detection', 'suffix': 'Finetune'},
 'experiment': {'suffix': 'Finetune', 'seed': 42}}

In [3]:
from main import run_pipeline
result = run_pipeline()

2026-01-15 11:31:38,486 - main - INFO - Starting training...
2026-01-15 11:31:38,487 - src.utils.config - INFO - Loaded config from configs/config.yaml
2026-01-15 11:31:38 - INFO - Starting training...
2026-01-15 11:31:38,488 - PCB_Detection - INFO - Starting training...
2026-01-15 11:31:38 - INFO - Seed fixed to 42
2026-01-15 11:31:38,490 - PCB_Detection - INFO - Seed fixed to 42
2026-01-15 11:31:38 - INFO - Using device: mps
2026-01-15 11:31:38,505 - PCB_Detection - INFO - Using device: mps
2026-01-15 11:31:38 - INFO - Loading model from checkpoint: weights/best_yolov11m_imgsz1280.pt
2026-01-15 11:31:38,506 - PCB_Detection - INFO - Loading model from checkpoint: weights/best_yolov11m_imgsz1280.pt
2026-01-15 11:31:38,507 - src.models.model_factory - INFO - Loading model from weights: weights/best_yolov11m_imgsz1280.pt
2026-01-15 11:31:38,548 - src.models.model_factory - INFO - Successfully loaded model from weights/best_yolov11m_imgsz1280.pt
2026-01-15 11:31:38 - INFO - Freezing first

2026-01-15 11:31:42,698 - src.utils.wandb_setup - INFO - W&B initialized: project=PCB_Detection, run_name=yolo11m-Finetune
2026-01-15 11:31:42 - INFO - Training parameters: {'data': '/Users/mungughyeon/내 드라이브/likelion/PCB/dataset/roboflow/data.yaml', 'epochs': 30, 'batch': 32, 'imgsz': 1280, 'patience': 10, 'device': device(type='mps'), 'workers': 8, 'project': '/Users/mungughyeon/내 드라이브/likelion/PCB/outputs', 'name': 'yolo11m-Finetune', 'val': True, 'optimizer': 'AdamW', 'lr0': 0.0002, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'mosaic': 0.1, 'mixup': 0.1, 'multi_scale': True}
2026-01-15 11:31:42,698 - PCB_Detection - INFO - Training parameters: {'data': '/Users/mungughyeon/내 드라이브/likelion/PCB/dataset/roboflow/data.yaml', 'epochs': 30, 'batch': 32, 'imgsz': 1280, 'patience': 10, 'device': device(type='mps'), 'workers': 8, 'project': '/Users/mungughyeon/내 드라이브/likelion/PCB/outputs', 'name': 'yolo11m-Finetune', 'val': True, 'optimizer': 'AdamW', 'lr0': 0.0002, 'lrf': 0.01, 

New https://pypi.org/project/ultralytics/8.4.2 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/mungughyeon/내 드라이브/likelion/PCB/dataset/roboflow/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=weights/best_yolov11m_imgsz1280.pt, momentum=0.937, mosaic=0.1, multi_scale=True, name=yolo11m-Finetune, nbs=64, nms=False, opset=None, optimize=F

2026-01-15 11:32:12 - ERROR - Training failed: MPS backend out of memory (MPS allocated: 27.03 GiB, other allocations: 1.88 GiB, max allowed: 30.19 GiB). Tried to allocate 1.88 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).
2026-01-15 11:32:12,064 - PCB_Detection - ERROR - Training failed: MPS backend out of memory (MPS allocated: 27.03 GiB, other allocations: 1.88 GiB, max allowed: 30.19 GiB). Tried to allocate 1.88 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).


RuntimeError: MPS backend out of memory (MPS allocated: 27.03 GiB, other allocations: 1.88 GiB, max allowed: 30.19 GiB). Tried to allocate 1.88 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).